# Stable DraCor workflow

```
services:
  api:
    image: ingoboerner/dracor-api:v0.86.3_local
    ports:
      - "8080:8080"
    depends_on:
      - metrics
      - fuseki
  metrics:
    image: ingoboerner/dracor-metrics:v1.2.1
    ports:
      - "8030:8030"
  frontend:
    image: ingoboerner/dracor-frontend:v1.4.3_local
    ports:
      - "8088:80"
    depends_on:
      - api
  fuseki:
    image: "stain/jena-fuseki"
    environment:
      - ADMIN_PASSWORD=qwerty
      - FUSEKI_DATASET_1=dracor
    ports:
      - "3030:3030"
    expose:
      - "3030"
```

In [1]:
!ls

docker-compose.empty.yml     stable-dracor-workflow.ipynb


Run 
`docker-compose -f docker-compose.empty.yml up`

Go to http://localhost:8088/

Should be an empty DraCor instance.

Functions to use the API:

In [3]:
# import libraries json and requests
import json
import requests

#corpusname:str -> []
def get(**kwargs):
    #corpusname=corpusname
    #playname=playname
    #apibase="https://dracor.org/api/"
    #method=method
    #parse_json: True
    
    #could set different apibase, e.g. https://staging.dracor.org/api/ [not recommended, pls use the production server]
    if "apibase" in kwargs:
        if kwargs["apibase"].endswith("/"):
            apibase = kwargs["apibase"]
        else:
            apibase = kwargs["apibase"] + "/"
    else:
        #use local API per default
        apibase = "http://localhost:8088/api/"
    if "corpusname" in kwargs and "playname" in kwargs:
        # used for /api/corpora/{corpusname}/play/{playname}/
        if "method" in kwargs:
            request_url = apibase + "corpora/" + kwargs["corpusname"] + "/play/" + kwargs["playname"] + "/" + kwargs["method"]
        else:
            request_url = apibase + "corpora/" + kwargs["corpusname"] + "/play/" + kwargs["playname"]
    elif "corpusname" in kwargs and not "playname" in kwargs:
        if "method" in kwargs:
            request_url = apibase + "corpora/" + kwargs["corpusname"] + "/" + kwargs["method"]
        else:
            request_url = apibase + "corpora/" + kwargs["corpusname"] 
    elif "method" in kwargs and not "corpusname" in kwargs and not "playname" in kwargs:
            request_url = apibase + kwargs["method"]
            
    else: 
        #nothing set
        request = request_url = apibase + "info"
    
    #send the response
    r = requests.get(request_url)
    if r.status_code == 200:
        #success!
        if "parse_json" in kwargs:
            if kwargs["parse_json"] == True:
                json_data = json.loads(r.text)
                return json_data
            else:
                return r.text
        else:
            return r.text
    else:
        raise Exception("Request was not successful. Server returned status code: "  + str(r.status_code))

Setup a corpus

In [7]:
new_corpus_name = "whatever"
new_corpus_title = "Whatever new corpus"

#needed for authorization
from requests.auth import HTTPBasicAuth

#Username of the local instance
usr = "admin"
#Password of the admin user
pwd = ""

#construct the payload
metadata = {
  "name": new_corpus_name,
  "title": new_corpus_title
}

#url of the corpora endpoint
corpora_endpoint_url = "http://localhost:8088/api/corpora"

#send the POST request using library requests
r = requests.post(corpora_endpoint_url, json = metadata, auth=HTTPBasicAuth(usr, pwd))

if r.status_code == 200:
    print("Success!" + " http://localhost:8088/" + new_corpus_name)

In [8]:
#check
get(method="corpora", parse_json=True)

[{'uri': 'https://dracor.org/api/corpora/whatever',
  'title': 'Whatever new corpus',
  'name': 'whatever',
  'acronym': 'WhateverDraCor'}]

In [9]:
#define the variables
corpus_repo_part = "gerdracor" # this is not the same as "corpusname"!
commit_id = "bfadf6b5844d4e05ea0501898a23c21f71c10cb3"
filename = "lessing-emilia-galotti.xml"

In [10]:
#concatinate to a download url
download_url = "https://raw.githubusercontent.com/dracor-org/" + corpus_repo_part + "/" + commit_id + "/tei/" + filename
print(download_url)

https://raw.githubusercontent.com/dracor-org/gerdracor/bfadf6b5844d4e05ea0501898a23c21f71c10cb3/tei/lessing-emilia-galotti.xml


In [11]:
r = requests.get(download_url)
#get only the text from the response and encode it in UTF-8 (important!)
tei = r.text.encode('utf-8')

In [12]:
#use split, to get everything before ".xml"
playname = filename.split(".xml")[0]

In [13]:
#construct the URL to use in the PUT request:
put_request_url = "http://localhost:8088/api/corpora/" + new_corpus_name + "/play/" + playname + "/tei"

In [14]:
# need the Content-Type to application/xml
headers = {'Content-Type': 'application/xml'}

In [15]:
r = requests.put(put_request_url, data=tei, headers=headers, auth=HTTPBasicAuth(usr, pwd))

Include whole corpora:

see also: https://github.com/dracor-org/vebidracor/blob/main/vebidracor-workflow.ipynb

```
corpora_to_include = [
    { 
        "corpusname": "ger",
        "repository": "https://github.com/dracor-org/gerdracor",
        "commit": "9135bd4598f54133f23df6edfc983b79f1616fb5",
        "exclude" : ["ger000480"]
    }
```

maybe also, but not implemented

```
"include" : ["abc", "xyz"]
```
